In [ ]:
# @title Cell 1: Library Setup & Dataset Path Configuration

"""
ASD Detection Project: Baseline XGBoost Implementation
Cell 1: Environment setup with cleaned train/val/test splits (ethnicity removed)
"""

print("="*80)
print("ASD DETECTION PROJECT: BASELINE XGBOOST IMPLEMENTATION")
print("Cell 1: Library Setup & Dataset Path Configuration")
print("="*80)

# ==========================================
# 1. GOOGLE DRIVE MOUNTING
# ==========================================

print("\n[DRIVE SETUP] Mounting Google Drive...")
print("-" * 50)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully")
except Exception as e:
    print(f"Drive mounting failed: {e}")
    raise

# ==========================================
# 2. PROJECT PATH CONFIGURATION & SETUP
# ==========================================

print("\n[PROJECT STRUCTURE] Validating and creating project paths...")
print("-" * 50)

import os

# Base project path
PROJECT_ROOT = '/content/drive/MyDrive/ASD_GWO_XGBoost_Project'

# Define project structure
PROJECT_PATHS = {
    'root': PROJECT_ROOT,
    'dataset': f"{PROJECT_ROOT}/01_Dataset",
    'splits': f"{PROJECT_ROOT}/01_Dataset/splits/no_ethnicity",
    'preprocessed': f"{PROJECT_ROOT}/01_Dataset/splits/no_ethnicity/preprocessed",
    'preprocessors': f"{PROJECT_ROOT}/01_Dataset/preprocessors",
    'results': f"{PROJECT_ROOT}/03_Results",
    'baseline_results': f"{PROJECT_ROOT}/03_Results/baseline_experiments"
}

# Create missing directories
print("Creating missing directories...")
for name, path in PROJECT_PATHS.items():
    if os.path.exists(path):
        print(f"Found {name}: {path}")
    else:
        os.makedirs(path, exist_ok=True)
        print(f"Created {name}: {path}")

# Define cleaned split file paths (ethnicity removed)
SPLIT_FILES = {
    'train': f"{PROJECT_PATHS['splits']}/train_set_clean.csv",
    'val': f"{PROJECT_PATHS['splits']}/val_set_clean.csv",
    'test': f"{PROJECT_PATHS['splits']}/test_set_clean.csv"
}

# Verify cleaned split files exist
print("\nVerifying cleaned split files...")
for split_name, file_path in SPLIT_FILES.items():
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / (1024**2)  # MB
        print(f"Found {split_name}_set_clean.csv: {file_size:.2f} MB")
    else:
        print(f"ERROR: Missing {split_name}_set_clean.csv")
        raise FileNotFoundError(f"Required file not found: {file_path}")

FILE_STATUS = "cleaned"
print(f"\nUsing cleaned split files (ethnicity removed)")

# ==========================================
# 3. LIBRARY IMPORTS
# ==========================================

print("\n[IMPORTS] Importing essential libraries...")
print("-" * 50)

# Core data processing
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, RFE, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.ensemble import RandomForestClassifier

# XGBoost
import xgboost as xgb

# Statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Utility libraries
import time
from datetime import datetime
import pickle

print("Essential libraries imported successfully")

# ==========================================
# 4. CONFIGURATION SETUP
# ==========================================

print("\n[CONFIGURATION] Setting up environment...")
print("-" * 50)

# Reproducibility configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)

# Visualization configuration
plt.style.use('default')
sns.set_palette("husl")

# Project configuration
CONFIG = {
    'random_state': RANDOM_STATE,
    'test_size': 0.2,
    'val_size': 0.16,
    'train_size': 0.64,
    'cv_folds': 5,
    'scoring_metric': 'roc_auc',
    'target_column': 'ASD_traits'
}

# XGBoost configuration for baseline
XGBOOST_CONFIG = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbosity': 0
}

print("Environment configured successfully")

# ==========================================
# 5. UTILITY FUNCTIONS
# ==========================================

print("\n[UTILITIES] Defining utility functions...")
print("-" * 50)

def create_results_directory(experiment_name: str) -> str:
    """Create and return results directory for experiment"""
    results_dir = f"{PROJECT_PATHS['baseline_results']}/{experiment_name}"
    os.makedirs(results_dir, exist_ok=True)
    return results_dir

def save_results(results: dict, filepath: str):
    """Save results dictionary to JSON file"""
    def json_serializer(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif pd.isna(obj):
            return None
        return str(obj)

    with open(filepath, 'w') as f:
        json.dump(results, f, indent=2, default=json_serializer)

def load_split_data(split_name: str) -> pd.DataFrame:
    """Load specific split data (train/val/test)"""
    file_path = SPLIT_FILES[split_name]
    return pd.read_csv(file_path)

def validate_data_consistency():
    """Validate that all splits have consistent columns and target distribution"""
    splits_info = {}

    for split_name in ['train', 'val', 'test']:
        try:
            df = load_split_data(split_name)
            splits_info[split_name] = {
                'shape': df.shape,
                'columns': list(df.columns),
                'target_dist': df[CONFIG['target_column']].value_counts().to_dict() if CONFIG['target_column'] in df.columns else None,
                'file_path': SPLIT_FILES[split_name]
            }
        except Exception as e:
            print(f"Error loading {split_name} set: {e}")
            splits_info[split_name] = {'error': str(e)}

    return splits_info

def assess_ethnicity_bias():
    """Assess whether ethnicity features are present (bias indicator)"""
    bias_assessment = {
        'ethnicity_features_found': [],
        'bias_risk': 'unknown',
        'recommendation': 'unknown'
    }

    try:
        # Check train set for ethnicity-related columns
        train_df = load_split_data('train')
        ethnicity_cols = [col for col in train_df.columns if 'ethnicity' in col.lower()]

        bias_assessment['ethnicity_features_found'] = ethnicity_cols

        if len(ethnicity_cols) > 0:
            bias_assessment['bias_risk'] = 'high'
            bias_assessment['recommendation'] = 'Run ethnicity removal script before proceeding'
            print(f"WARNING: {len(ethnicity_cols)} ethnicity-related features detected")
            print(f"Features: {ethnicity_cols}")
        else:
            bias_assessment['bias_risk'] = 'low'
            bias_assessment['recommendation'] = 'Proceed with current dataset'
            print("No ethnicity-related features detected")

    except Exception as e:
        bias_assessment['error'] = str(e)
        print(f"Bias assessment failed: {e}")

    return bias_assessment

def create_setup_summary():
    """Create comprehensive setup summary"""
    return {
        'timestamp': datetime.now().isoformat(),
        'project_paths': PROJECT_PATHS,
        'split_files': SPLIT_FILES,
        'file_status': FILE_STATUS,
        'configuration': CONFIG,
        'xgboost_config': XGBOOST_CONFIG,
        'package_versions': packages,
        'splits_info': SPLITS_INFO if 'SPLITS_INFO' in globals() else {},
        'bias_assessment': BIAS_ASSESSMENT if 'BIAS_ASSESSMENT' in globals() else {},
        'cell_status': 'completed'
    }

print("Utility functions defined successfully")

# ==========================================
# 6. SYSTEM VALIDATION & DATA ASSESSMENT
# ==========================================

print("\n[VALIDATION] System validation and data assessment...")
print("-" * 50)

# Check Python version
import sys
print(f"Python version: {sys.version.split()[0]}")

# Check key package versions
packages = {
    'pandas': pd.__version__,
    'numpy': np.__version__,
    'scikit-learn': __import__('sklearn').__version__,
    'xgboost': xgb.__version__
}

print("Package versions:")
for pkg, version in packages.items():
    print(f"  {pkg}: {version}")

# Validate GPU availability
try:
    import GPUtil
    gpus = GPUtil.getGPUs()
    if gpus:
        print(f"GPU available: {gpus[0].name}")
    else:
        print("No GPU detected")
except:
    print("GPU utilities not available")

# Data consistency and bias assessment
print("\nPerforming data assessment...")
try:
    splits_info = validate_data_consistency()

    # Check for ethnicity bias indicators
    bias_assessment = assess_ethnicity_bias()

    print("Data assessment completed")

    # Store for next cell
    globals()['SPLITS_INFO'] = splits_info
    globals()['BIAS_ASSESSMENT'] = bias_assessment

except Exception as e:
    print(f"Data assessment failed: {e}")
    raise

# ==========================================
# 7. EXPORT SETUP INFORMATION
# ==========================================

print("\n[EXPORT] Saving setup information...")
print("-" * 50)

# Create baseline results directory
os.makedirs(PROJECT_PATHS['baseline_results'], exist_ok=True)

# Setup summary
setup_info = create_setup_summary()

# Save setup info
setup_path = f"{PROJECT_PATHS['baseline_results']}/setup_info.json"
save_results(setup_info, setup_path)

print(f"Setup information saved: {setup_path}")

# ==========================================
# 8. COMPLETION SUMMARY
# ==========================================

print("\n[SUMMARY] Cell 1 Setup Summary")
print("="*80)

print("CONFIGURATION STATUS:")
print(f"  Project root: {PROJECT_ROOT}")
print(f"  File status: {FILE_STATUS}")

if 'SPLITS_INFO' in globals():
    for split_name, info in SPLITS_INFO.items():
        if 'shape' in info:
            print(f"  {split_name} set: {info['shape'][0]} samples x {info['shape'][1]} features")
        else:
            print(f"  {split_name} set: Error loading")

print(f"  Target column: {CONFIG['target_column']}")
print(f"  Random state: {CONFIG['random_state']}")

print("\nPACKAGE STATUS:")
for pkg, version in packages.items():
    print(f"  {pkg}: {version}")

if 'BIAS_ASSESSMENT' in globals():
    print(f"\nBIAS ASSESSMENT:")
    print(f"  Ethnicity features detected: {len(BIAS_ASSESSMENT.get('ethnicity_features_found', []))}")
    print(f"  Bias risk: {BIAS_ASSESSMENT.get('bias_risk', 'unknown')}")
    print(f"  Recommendation: {BIAS_ASSESSMENT.get('recommendation', 'unknown')}")

print("\nFILE AVAILABILITY:")
print("  Using ethnicity-removed split files")
print("  Ready for bias-free analysis")

print("\nNEXT STEPS:")
print("  Cell 2: Load cleaned splits, perform EDA, and generate preprocessed CSV files")
print("  Cell 3: Baseline 1 - XGBoost with all features (using preprocessed data)")
print("  Cell 4: Baseline 2 - XGBoost with feature selection (using preprocessed data)")

print("\n" + "="*80)
print("CELL 1 COMPLETED SUCCESSFULLY")
print("Variables and paths ready for Cell 2")
print("="*80)

ASD DETECTION PROJECT: BASELINE XGBOOST IMPLEMENTATION
Cell 1: Library Setup & Dataset Path Configuration

[DRIVE SETUP] Mounting Google Drive...
--------------------------------------------------
Mounted at /content/drive
Google Drive mounted successfully

[PROJECT STRUCTURE] Validating and creating project paths...
--------------------------------------------------
Creating missing directories...
Found root: /content/drive/MyDrive/ASD_GWO_XGBoost_Project
Found dataset: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset
Found splits: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset/splits/no_ethnicity
Found preprocessed: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset/splits/no_ethnicity/preprocessed
Created preprocessors: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset/preprocessors
Found results: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results
Found baseline_results: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/baseline_expe

In [ ]:
# @title Cell 2: EDA & Preprocessing Pipeline

"""
ASD Detection Project: Baseline XGBoost Implementation
Cell 2: Load cleaned splits, perform publication-ready EDA, and generate preprocessed CSV files
Enhanced for IEEE conference standard with comprehensive clinical analysis
"""

print("="*80)
print("ASD DETECTION PROJECT: BASELINE XGBOOST IMPLEMENTATION")
print("Cell 2: Enhanced EDA & Preprocessing Pipeline (Publication-Ready)")
print("="*80)

# ==========================================
# 1. VERIFY CELL 1 EXECUTION
# ==========================================

print("\n[VERIFICATION] Checking Cell 1 completion...")
print("-" * 50)

try:
    # Verify essential variables from Cell 1
    PROJECT_PATHS
    SPLIT_FILES
    CONFIG
    XGBOOST_CONFIG
    SPLITS_INFO
    print("Cell 1 variables loaded successfully")
except NameError as e:
    print(f"Error: {e}")
    print("Please run Cell 1 first")
    raise Exception("Cell 1 must be executed before Cell 2")

# ==========================================
# 2. LOAD CLEANED SPLIT DATASETS
# ==========================================

print("\n[DATA LOADING] Loading cleaned split datasets...")
print("-" * 50)

def load_all_splits():
    """Load all three cleaned split datasets"""
    splits = {}

    for split_name in ['train', 'val', 'test']:
        try:
            df = load_split_data(split_name)
            splits[split_name] = df
            print(f"Loaded {split_name} set: {df.shape[0]:,} samples x {df.shape[1]} features")

            # Verify target column exists
            if CONFIG['target_column'] not in df.columns:
                raise ValueError(f"Target column '{CONFIG['target_column']}' not found in {split_name} set")

        except Exception as e:
            print(f"Error loading {split_name} set: {e}")
            raise

    return splits

# Load all splits
SPLITS_DATA = load_all_splits()

# Separate features and targets for each split
def prepare_feature_target_splits(splits_data, target_col):
    """Separate features and targets for all splits"""
    prepared_splits = {}

    for split_name, df in splits_data.items():
        X = df.drop(columns=[target_col])
        y = df[target_col]
        prepared_splits[split_name] = {'X': X, 'y': y, 'df': df}

    return prepared_splits

PREPARED_SPLITS = prepare_feature_target_splits(SPLITS_DATA, CONFIG['target_column'])

print(f"\nFeature-target separation completed")
print(f"Feature columns: {PREPARED_SPLITS['train']['X'].shape[1]}")

# ==========================================
# 3. PUBLICATION-READY EDA ANALYSIS
# ==========================================

print("\n[PUBLICATION EDA] Comprehensive clinical analysis (train set only)...")
print("-" * 50)

def comprehensive_clinical_eda():
    """Publication-ready EDA with clinical focus and statistical rigor"""

    train_df = PREPARED_SPLITS['train']['df']
    X_train = PREPARED_SPLITS['train']['X']
    y_train = PREPARED_SPLITS['train']['y']

    clinical_eda = {
        'dataset_overview': {},
        'clinical_target_analysis': {},
        'behavioral_features_analysis': {},
        'clinical_assessment_analysis': {},
        'demographic_analysis': {},
        'statistical_tests': {}
    }

    # Section 1: Dataset Overview & Quality Assessment
    print("Section 1: Dataset Overview & Quality Assessment")

    # Basic demographics
    dataset_overview = {
        'total_samples': len(train_df),
        'total_features': len(X_train.columns),
        'memory_usage_mb': train_df.memory_usage(deep=True).sum() / 1024**2,
        'data_quality': {
            'missing_values': train_df.isnull().sum().sum(),
            'duplicate_rows': train_df.duplicated().sum()
        }
    }

    clinical_eda['dataset_overview'] = dataset_overview

    # Section 2: Clinical Target Analysis (Priority Section)
    print("Section 2: Clinical Target Analysis")

    target_analysis = {}

    # ASD prevalence across splits
    prevalence_by_split = {}
    for split_name in ['train', 'val', 'test']:
        split_y = PREPARED_SPLITS[split_name]['y']
        target_dist = split_y.value_counts()
        target_props = split_y.value_counts(normalize=True)

        prevalence_by_split[split_name] = {
            'total_samples': len(split_y),
            'asd_count': target_dist.get(1, 0),
            'non_asd_count': target_dist.get(0, 0),
            'asd_prevalence': target_props.get(1, 0),
            'balance_ratio': min(target_dist) / max(target_dist) if len(target_dist) > 1 else 1.0
        }

    target_analysis['prevalence_by_split'] = prevalence_by_split

    # Age-stratified analysis if Age column exists
    age_cols = [col for col in X_train.columns if 'age' in col.lower()]
    if age_cols:
        age_col = age_cols[0]
        age_stratified = {}

        # Define age groups for pediatric population
        age_groups = pd.cut(X_train[age_col], bins=[0, 3, 6, 10, 15, 18],
                           labels=['0-3', '4-6', '7-10', '11-15', '16-18'])

        for age_group in age_groups.cat.categories:
            mask = age_groups == age_group
            if mask.sum() > 0:
                group_y = y_train[mask]
                if len(group_y) > 0:
                    asd_rate = (group_y == 1).mean()
                    age_stratified[age_group] = {
                        'total': len(group_y),
                        'asd_count': (group_y == 1).sum(),
                        'asd_rate': asd_rate
                    }

        target_analysis['age_stratified'] = age_stratified

    clinical_eda['clinical_target_analysis'] = target_analysis

    # Section 3: Behavioral Features Analysis (A1-A10) - Priority Section
    print("Section 3: Behavioral Features Analysis (A1-A10)")

    behavioral_analysis = {}

    # Identify A1-A10 features
    behavioral_features = [col for col in X_train.columns if
                          col.startswith('A') and len(col) <= 3 and
                          any(char.isdigit() for char in col)]
    behavioral_features.sort()

    if behavioral_features:
        behavioral_stats = {}

        for feature in behavioral_features:
            feature_data = X_train[feature]

            # Basic statistics by ASD status
            asd_group = feature_data[y_train == 1]
            non_asd_group = feature_data[y_train == 0]

            # Effect size calculation (Cohen's d)
            def cohens_d(group1, group2):
                n1, n2 = len(group1), len(group2)
                s1, s2 = group1.std(), group2.std()
                s_pooled = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
                return (group1.mean() - group2.mean()) / s_pooled if s_pooled != 0 else 0

            # Statistical test
            from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu

            stat_test_result = {}
            try:
                if feature_data.nunique() <= 2:  # Binary feature
                    contingency = pd.crosstab(y_train, feature_data)
                    if contingency.size > 1 and contingency.min().min() > 0:  # Avoid cells with 0
                        chi2, p_val, _, _ = chi2_contingency(contingency)
                        stat_test_result = {
                            'test': 'chi_square',
                            'statistic': float(chi2),
                            'p_value': float(p_val),
                            'significant': p_val < 0.05
                        }
                    else:
                        stat_test_result = {
                            'test': 'chi_square',
                            'error': 'Insufficient data for test',
                            'significant': False
                        }
                else:  # Continuous feature
                    if len(asd_group) > 0 and len(non_asd_group) > 0:
                        # Check normality assumption (simplified)
                        if len(asd_group) >= 30 and len(non_asd_group) >= 30:
                            stat, p_val = ttest_ind(asd_group, non_asd_group)
                            test_name = 't_test'
                        else:
                            stat, p_val = mannwhitneyu(asd_group, non_asd_group)
                            test_name = 'mann_whitney'

                        stat_test_result = {
                            'test': test_name,
                            'statistic': float(stat),
                            'p_value': float(p_val),
                            'significant': p_val < 0.05
                        }
                    else:
                        stat_test_result = {
                            'test': 'insufficient_data',
                            'error': 'Empty groups',
                            'significant': False
                        }
            except Exception as e:
                stat_test_result = {
                    'test': 'failed',
                    'error': str(e),
                    'significant': False
                }

            behavioral_stats[feature] = {
                'asd_mean': asd_group.mean() if len(asd_group) > 0 else 0,
                'non_asd_mean': non_asd_group.mean() if len(non_asd_group) > 0 else 0,
                'asd_std': asd_group.std() if len(asd_group) > 0 else 0,
                'non_asd_std': non_asd_group.std() if len(non_asd_group) > 0 else 0,
                'cohens_d': cohens_d(asd_group, non_asd_group) if len(asd_group) > 0 and len(non_asd_group) > 0 else 0,
                'statistical_test': stat_test_result
            }

        # Behavioral pattern analysis
        behavioral_correlation = X_train[behavioral_features].corr()
        behavioral_total_score = X_train[behavioral_features].sum(axis=1)

        behavioral_analysis = {
            'features_analyzed': behavioral_features,
            'individual_statistics': behavioral_stats,
            'internal_correlation_mean': behavioral_correlation.values[np.triu_indices_from(behavioral_correlation.values, k=1)].mean(),
            'total_score_stats': {
                'asd_mean': behavioral_total_score[y_train == 1].mean(),
                'non_asd_mean': behavioral_total_score[y_train == 0].mean(),
                'asd_std': behavioral_total_score[y_train == 1].std(),
                'non_asd_std': behavioral_total_score[y_train == 0].std()
            }
        }

    clinical_eda['behavioral_features_analysis'] = behavioral_analysis

    # Section 4: Clinical Assessment Scales - Priority Section
    print("Section 4: Clinical Assessment Scales")

    assessment_analysis = {}

    # Identify clinical scales
    scale_features = {
        'CARS': [col for col in X_train.columns if 'CARS' in col or 'Rating' in col],
        'SRS': [col for col in X_train.columns if 'SRS' in col or 'Social' in col and 'Scale' in col],
        'AQ10': [col for col in X_train.columns if 'AQ' in col or 'Qchat' in col or 'Score' in col]
    }

    for scale_name, scale_cols in scale_features.items():
        if scale_cols:
            scale_col = scale_cols[0]  # Take first matching column
            scale_data = X_train[scale_col]

            # Distribution analysis by ASD status
            asd_scores = scale_data[y_train == 1]
            non_asd_scores = scale_data[y_train == 0]

            # Statistical comparison
            if len(asd_scores) > 0 and len(non_asd_scores) > 0:
                stat_result = {}
                try:
                    if len(asd_scores) >= 30 and len(non_asd_scores) >= 30:
                        stat, p_val = ttest_ind(asd_scores, non_asd_scores)
                        test_name = 't_test'
                    else:
                        stat, p_val = mannwhitneyu(asd_scores, non_asd_scores)
                        test_name = 'mann_whitney'

                    stat_result = {
                        'test': test_name,
                        'statistic': stat,
                        'p_value': p_val,
                        'significant': p_val < 0.05
                    }
                except:
                    stat_result = {'test': 'failed', 'error': 'Insufficient data'}

                assessment_analysis[scale_name] = {
                    'column': scale_col,
                    'asd_mean': asd_scores.mean(),
                    'non_asd_mean': non_asd_scores.mean(),
                    'asd_std': asd_scores.std(),
                    'non_asd_std': non_asd_scores.std(),
                    'statistical_test': stat_result,
                    'score_range': {
                        'min': scale_data.min(),
                        'max': scale_data.max(),
                        'unique_values': scale_data.nunique()
                    }
                }

    clinical_eda['clinical_assessment_analysis'] = assessment_analysis

    # Section 5: Demographic Analysis
    print("Section 5: Demographic Analysis")

    demographic_analysis = {}

    # Gender analysis
    gender_cols = [col for col in X_train.columns if any(term in col.lower() for term in ['sex', 'gender'])]
    if gender_cols:
        gender_col = gender_cols[0]
        gender_crosstab = pd.crosstab(X_train[gender_col], y_train)

        # Chi-square test for gender association
        try:
            chi2, p_val, _, _ = chi2_contingency(gender_crosstab)
            gender_test = {
                'test': 'chi_square',
                'statistic': chi2,
                'p_value': p_val,
                'significant': p_val < 0.05
            }
        except:
            gender_test = {'test': 'failed', 'error': 'Insufficient data'}

        demographic_analysis['gender'] = {
            'column': gender_col,
            'crosstab': gender_crosstab.to_dict(),
            'statistical_test': gender_test
        }

    clinical_eda['demographic_analysis'] = demographic_analysis

    return clinical_eda

# Perform comprehensive clinical EDA
CLINICAL_EDA = comprehensive_clinical_eda()

# ==========================================
# 4. ENHANCED REDUNDANCY ANALYSIS
# ==========================================

print("\n[REDUNDANCY ANALYSIS] Comprehensive feature redundancy assessment...")
print("-" * 50)

def comprehensive_redundancy_analysis(X_train, correlation_threshold=0.95, variance_threshold=0.01):
    """Comprehensive redundancy analysis with clinical considerations"""

    redundancy_report = {
        'correlation_analysis': {},
        'variance_analysis': {},
        'duplicate_analysis': {},
        'clinical_analysis': {}
    }

    # 1. Enhanced correlation analysis with data type safety
    print("Step 1: Correlation analysis")

    # Get truly numeric columns
    numeric_cols = []
    for col in X_train.columns:
        if pd.api.types.is_numeric_dtype(X_train[col]):
            try:
                pd.to_numeric(X_train[col], errors='raise')
                numeric_cols.append(col)
            except (ValueError, TypeError):
                print(f"  Skipping {col}: contains non-numeric values")
        else:
            # Check if it's a binary categorical that can be safely converted
            unique_vals = X_train[col].dropna().unique()
            if len(unique_vals) <= 2 and all(val in ['Yes', 'No', 'yes', 'no', '1', '0', 1, 0, True, False] for val in unique_vals):
                print(f"  Converting binary categorical {col} for analysis")
                try:
                    temp_col = X_train[col].map({'Yes': 1, 'No': 0, 'yes': 1, 'no': 0, '1': 1, '0': 0, 1: 1, 0: 0, True: 1, False: 0})
                    if not temp_col.isnull().all():
                        numeric_cols.append(col)
                except:
                    print(f"  Skipping {col}: conversion failed")

    print(f"  Analyzing {len(numeric_cols)} numeric/binary features for correlation")

    if len(numeric_cols) > 1:
        corr_data = X_train[numeric_cols].copy()

        # Convert binary categorical columns for correlation analysis
        for col in corr_data.columns:
            if not pd.api.types.is_numeric_dtype(corr_data[col]):
                corr_data[col] = corr_data[col].map({'Yes': 1, 'No': 0, 'yes': 1, 'no': 0})

        try:
            corr_matrix = corr_data.corr().abs()

            # Find high correlation pairs
            high_corr_pairs = []
            for i in range(len(corr_matrix.columns)):
                for j in range(i+1, len(corr_matrix.columns)):
                    corr_val = corr_matrix.iloc[i, j]
                    if not np.isnan(corr_val) and corr_val > correlation_threshold:
                        high_corr_pairs.append({
                            'feature1': corr_matrix.columns[i],
                            'feature2': corr_matrix.columns[j],
                            'correlation': float(corr_val)
                        })

            print(f"  Found {len(high_corr_pairs)} high correlation pairs (>{correlation_threshold})")
            redundancy_report['correlation_analysis'] = {
                'threshold': correlation_threshold,
                'high_corr_pairs': high_corr_pairs,
                'correlation_matrix': corr_matrix,
                'numeric_columns': numeric_cols
            }
        except Exception as e:
            print(f"  Correlation analysis failed: {e}")
            redundancy_report['correlation_analysis'] = {
                'error': str(e),
                'threshold': correlation_threshold,
                'high_corr_pairs': [],
                'numeric_columns': []
            }

    # 2. Variance analysis
    print("Step 2: Low variance analysis")
    low_variance_features = []

    for col in numeric_cols:
        try:
            col_data = X_train[col]
            if not pd.api.types.is_numeric_dtype(col_data):
                col_data = col_data.map({'Yes': 1, 'No': 0, 'yes': 1, 'no': 0})

            variance = col_data.var()
            if not np.isnan(variance) and variance < variance_threshold:
                low_variance_features.append({
                    'feature': col,
                    'variance': float(variance)
                })
        except Exception as e:
            print(f"  Skipping variance analysis for {col}: {e}")

    print(f"  Found {len(low_variance_features)} low variance features (<{variance_threshold})")
    redundancy_report['variance_analysis'] = {
        'threshold': variance_threshold,
        'low_variance_features': low_variance_features
    }

    # 3. Exact duplicate detection
    print("Step 3: Duplicate feature detection")
    duplicate_features = []

    try:
        for i in range(len(X_train.columns)):
            for j in range(i+1, len(X_train.columns)):
                col1, col2 = X_train.columns[i], X_train.columns[j]
                if X_train[col1].equals(X_train[col2]):
                    duplicate_features.append({
                        'feature1': col1,
                        'feature2': col2
                    })

        print(f"  Found {len(duplicate_features)} exact duplicate feature pairs")
    except Exception as e:
        print(f"  Duplicate detection failed: {e}")

    redundancy_report['duplicate_analysis'] = {
        'exact_duplicates': duplicate_features
    }

    # 4. Enhanced clinical grouping analysis
    print("Step 4: Clinical feature grouping")

    clinical_groups = {
        'behavioral_core': [col for col in X_train.columns if col.startswith('A') and len(col) >= 2 and col[1:2].isdigit()],
        'assessment_scales': [col for col in X_train.columns if any(scale in col for scale in ['Scale', 'Score', 'Rating', 'Qchat'])],
        'demographic': [col for col in X_train.columns if any(demo in col.lower() for demo in ['age', 'sex', 'gender'])],
        'medical_history': [col for col in X_train.columns if any(med in col.lower() for med in ['disorder', 'delay', 'depression', 'anxiety', 'jaundice'])]
    }

    # Analyze within-group correlations for numeric groups only
    group_correlations = {}
    for group_name, group_features in clinical_groups.items():
        if len(group_features) > 1:
            group_numeric = [col for col in group_features if col in numeric_cols]

            if len(group_numeric) > 1:
                try:
                    group_data = X_train[group_numeric].copy()

                    # Convert binary categorical if needed
                    for col in group_data.columns:
                        if not pd.api.types.is_numeric_dtype(group_data[col]):
                            group_data[col] = group_data[col].map({'Yes': 1, 'No': 0, 'yes': 1, 'no': 0})

                    group_corr = group_data.corr().abs()
                    upper_triangle = group_corr.where(np.triu(np.ones(group_corr.shape), k=1).astype(bool))
                    avg_corr = upper_triangle.stack().mean()

                    if not np.isnan(avg_corr):
                        group_correlations[group_name] = {
                            'features': group_features,
                            'numeric_features': group_numeric,
                            'count': len(group_features),
                            'numeric_count': len(group_numeric),
                            'avg_internal_correlation': float(avg_corr),
                            'correlation_matrix': group_corr
                        }
                        print(f"  {group_name}: {len(group_features)} features ({len(group_numeric)} numeric), avg correlation: {avg_corr:.3f}")
                except Exception as e:
                    print(f"  {group_name}: correlation analysis error: {e}")

    redundancy_report['clinical_analysis'] = {
        'clinical_groups': clinical_groups,
        'group_correlations': group_correlations
    }

    return redundancy_report

# Perform enhanced redundancy analysis
REDUNDANCY_REPORT = comprehensive_redundancy_analysis(PREPARED_SPLITS['train']['X'])

# ==========================================
# 5. PREPROCESSING PIPELINE IMPLEMENTATION
# ==========================================

print("\n[PREPROCESSING] Implementing fit-transform pipeline...")
print("-" * 50)

def create_preprocessing_pipeline(X_train, y_train):
    """Create preprocessing pipeline fitted on train data only"""
    pipeline_components = {}

    # Identify feature types
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

    print(f"Step 1: Feature type identification")
    print(f"  Numeric columns: {len(numeric_cols)}")
    print(f"  Categorical columns: {len(categorical_cols)}")

    # 1. Missing value imputation
    print("Step 2: Missing value imputation")

    if numeric_cols:
        numeric_imputer = SimpleImputer(strategy='median')
        numeric_imputer.fit(X_train[numeric_cols])
        pipeline_components['numeric_imputer'] = numeric_imputer
        print(f"  Numeric imputer fitted (median strategy)")

    if categorical_cols:
        categorical_imputer = SimpleImputer(strategy='most_frequent')
        categorical_imputer.fit(X_train[categorical_cols])
        pipeline_components['categorical_imputer'] = categorical_imputer
        print(f"  Categorical imputer fitted (mode strategy)")

    # 2. Categorical encoding
    print("Step 3: Categorical encoding")

    if categorical_cols:
        label_encoders = {}
        for col in categorical_cols:
            le = LabelEncoder()
            le.fit(X_train[col].astype(str))
            label_encoders[col] = le

        pipeline_components['label_encoders'] = label_encoders
        print(f"  Label encoders fitted for {len(label_encoders)} categorical features")

    # 3. Feature scaling
    print("Step 4: Feature scaling")

    # Identify binary features from EDA results
    binary_features = []
    for col in numeric_cols:
        unique_vals = X_train[col].dropna().unique()
        if len(unique_vals) == 2 and set(unique_vals).issubset({0, 1, 0.0, 1.0, True, False}):
            binary_features.append(col)

    scale_cols = [col for col in numeric_cols if col not in binary_features]

    if scale_cols:
        scaler = StandardScaler()
        scaler.fit(X_train[scale_cols])
        pipeline_components['scaler'] = scaler
        pipeline_components['scale_columns'] = scale_cols
        print(f"  Scaler fitted for {len(scale_cols)} continuous features")

    # 4. Target encoding
    print("Step 5: Target encoding")

    if y_train.dtype == 'object':
        target_encoder = LabelEncoder()
        target_encoder.fit(y_train)
        pipeline_components['target_encoder'] = target_encoder
        print(f"  Target encoder fitted")

    pipeline_components['feature_columns'] = {
        'numeric': numeric_cols,
        'categorical': categorical_cols,
        'scale': scale_cols,
        'binary': binary_features
    }

    return pipeline_components

def apply_preprocessing_pipeline(X, y, pipeline_components, split_name):
    """Apply preprocessing pipeline to any split"""
    X_processed = X.copy()
    y_processed = y.copy()

    print(f"Applying preprocessing to {split_name} set...")

    # 1. Apply missing value imputation
    if 'numeric_imputer' in pipeline_components:
        numeric_cols = pipeline_components['feature_columns']['numeric']
        X_processed[numeric_cols] = pipeline_components['numeric_imputer'].transform(X_processed[numeric_cols])

    if 'categorical_imputer' in pipeline_components:
        categorical_cols = pipeline_components['feature_columns']['categorical']
        X_processed[categorical_cols] = pipeline_components['categorical_imputer'].transform(X_processed[categorical_cols])

    # 2. Apply categorical encoding
    if 'label_encoders' in pipeline_components:
        for col, encoder in pipeline_components['label_encoders'].items():
            X_processed[col] = encoder.transform(X_processed[col].astype(str))

    # 3. Apply feature scaling
    if 'scaler' in pipeline_components:
        scale_cols = pipeline_components['scale_columns']
        X_processed[scale_cols] = pipeline_components['scaler'].transform(X_processed[scale_cols])

    # 4. Apply target encoding
    if 'target_encoder' in pipeline_components:
        y_processed = pipeline_components['target_encoder'].transform(y_processed)

    print(f"  {split_name} preprocessing completed: {X_processed.shape}")

    return X_processed, y_processed

# Create preprocessing pipeline
PREPROCESSING_PIPELINE = create_preprocessing_pipeline(
    PREPARED_SPLITS['train']['X'],
    PREPARED_SPLITS['train']['y']
)

# Apply preprocessing to all splits
PROCESSED_SPLITS = {}
for split_name in ['train', 'val', 'test']:
    X_processed, y_processed = apply_preprocessing_pipeline(
        PREPARED_SPLITS[split_name]['X'],
        PREPARED_SPLITS[split_name]['y'],
        PREPROCESSING_PIPELINE,
        split_name
    )
    PROCESSED_SPLITS[split_name] = {
        'X': X_processed,
        'y': y_processed
    }

print("Preprocessing pipeline completed for all splits")

# ==========================================
# 6. SAVE PREPROCESSED DATA TO CSV
# ==========================================

print("\n[SAVE PREPROCESSED] Saving preprocessed datasets to CSV...")
print("-" * 50)

# Create preprocessed directory
preprocessed_dir = PROJECT_PATHS['preprocessed']
os.makedirs(preprocessed_dir, exist_ok=True)

# Save each preprocessed split
for split_name in ['train', 'val', 'test']:
    X_processed = PROCESSED_SPLITS[split_name]['X']
    y_processed = PROCESSED_SPLITS[split_name]['y']

    # Combine features and target
    processed_df = X_processed.copy()
    processed_df[CONFIG['target_column']] = y_processed

    # Save to CSV
    output_path = f"{preprocessed_dir}/{split_name}_set_preprocessed.csv"
    processed_df.to_csv(output_path, index=False)

    file_size = os.path.getsize(output_path) / (1024**2)  # MB
    print(f"Saved {split_name}_set_preprocessed.csv: {file_size:.2f} MB ({processed_df.shape[0]} rows x {processed_df.shape[1]} cols)")

# Save preprocessing pipeline objects
preprocessors_dir = PROJECT_PATHS['preprocessors']
os.makedirs(preprocessors_dir, exist_ok=True)

# Save label encoders
if 'label_encoders' in PREPROCESSING_PIPELINE:
    with open(f"{preprocessors_dir}/label_encoders.pkl", 'wb') as f:
        pickle.dump(PREPROCESSING_PIPELINE['label_encoders'], f)
    print(f"Saved label_encoders.pkl")

# Save scaler
if 'scaler' in PREPROCESSING_PIPELINE:
    with open(f"{preprocessors_dir}/scaler.pkl", 'wb') as f:
        pickle.dump(PREPROCESSING_PIPELINE['scaler'], f)
    print(f"Saved scaler.pkl")

# Save preprocessing configuration
preprocessing_config = {
    'feature_columns': PREPROCESSING_PIPELINE['feature_columns'],
    'preprocessing_steps': list(PREPROCESSING_PIPELINE.keys()),
    'timestamp': datetime.now().isoformat(),
    'random_state': CONFIG['random_state']
}

with open(f"{preprocessors_dir}/preprocessing_config.json", 'w') as f:
    json.dump(preprocessing_config, f, indent=2)
print(f"Saved preprocessing_config.json")

print(f"\nAll preprocessed files saved to: {preprocessed_dir}")

# ==========================================
# 7. QUALITY ASSESSMENT
# ==========================================

print("\n[QUALITY ASSESSMENT] Post-preprocessing quality evaluation...")
print("-" * 50)

def assess_preprocessing_quality():
    """Assess quality of preprocessing results"""
    quality_report = {}

    for split_name in ['train', 'val', 'test']:
        X = PROCESSED_SPLITS[split_name]['X']
        y = PROCESSED_SPLITS[split_name]['y']

        missing_count = X.isnull().sum().sum()
        numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        inf_count = np.isinf(X.select_dtypes(include=[np.number])).sum().sum()

        split_quality = {
            'shape': X.shape,
            'missing_values': int(missing_count),
            'infinite_values': int(inf_count),
            'numeric_features': len(numeric_cols),
            'target_encoded': not isinstance(y.iloc[0], str) if len(y) > 0 else True
        }

        quality_report[split_name] = split_quality

        print(f"{split_name.upper()} quality:")
        print(f"  Shape: {X.shape}")
        print(f"  Missing values: {missing_count}")
        print(f"  Infinite values: {inf_count}")

    return quality_report

QUALITY_REPORT = assess_preprocessing_quality()

# ==========================================
# 8. ENHANCED PUBLICATION-READY VISUALIZATIONS
# ==========================================

print("\n[VISUALIZATIONS] Generating publication-ready analysis plots...")
print("-" * 50)

def generate_clinical_eda_visualization():
    """Generate comprehensive clinical EDA visualization"""
    results_dir = create_results_directory("eda_analysis")

    # Set publication-ready style
    plt.style.use('default')
    plt.rcParams.update({
        'figure.dpi': 300,
        'font.size': 9,
        'axes.titlesize': 11,
        'axes.labelsize': 9,
        'xtick.labelsize': 8,
        'ytick.labelsize': 8,
        'legend.fontsize': 8,
        'figure.titlesize': 13
    })

    # Create comprehensive clinical analysis plot with proper spacing
    fig = plt.figure(figsize=(18, 14))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3,
                         left=0.08, right=0.95, top=0.93, bottom=0.08)

    # Section 1: Target distributions across splits
    for idx, split_name in enumerate(['train', 'val', 'test']):
        ax = fig.add_subplot(gs[0, idx])
        y = PROCESSED_SPLITS[split_name]['y']
        target_counts = pd.Series(y).value_counts().sort_index()

        colors = ['#2E86AB', '#A23B72']
        bars = ax.bar(['Non-ASD', 'ASD'], target_counts.values, color=colors, alpha=0.8)
        ax.set_title(f'{split_name.title()} Set Distribution\n(n={len(y):,})',
                    fontweight='bold', pad=10)
        ax.set_ylabel('Count')

        # Add statistical annotations with proper positioning
        for bar, count in zip(bars, target_counts.values):
            percentage = (count / len(y)) * 100
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(target_counts)*0.02,
                   f'{count:,}\n({percentage:.1f}%)', ha='center', va='bottom',
                   fontweight='bold', fontsize=8)

        # Set proper y-axis limits to prevent overflow
        ax.set_ylim(0, max(target_counts) * 1.15)

    # Section 2: Behavioral features analysis
    behavioral_data = CLINICAL_EDA['behavioral_features_analysis']
    if 'individual_statistics' in behavioral_data and behavioral_data['individual_statistics']:
        ax = fig.add_subplot(gs[1, :])

        features = list(behavioral_data['individual_statistics'].keys())
        cohens_d_values = [behavioral_data['individual_statistics'][f]['cohens_d'] for f in features]
        p_values = [behavioral_data['individual_statistics'][f]['statistical_test'].get('p_value', 1.0) for f in features]

        # Color bars by significance
        colors = ['#FF6B6B' if p < 0.05 else '#87CEEB' for p in p_values]
        bars = ax.bar(features, cohens_d_values, color=colors, alpha=0.8)

        ax.set_title('Behavioral Features Effect Sizes (Cohen\'s d)\nRed: Significant (p<0.05), Blue: Non-significant',
                    fontweight='bold', pad=15)
        ax.set_xlabel('Behavioral Features (A1-A10)')
        ax.set_ylabel('Cohen\'s d')

        # Add reference lines
        ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Medium Effect')
        ax.axhline(y=0.8, color='black', linestyle='--', alpha=0.7, label='Large Effect')
        ax.legend(loc='upper right')

        # Add significance annotations with proper positioning
        max_val = max(cohens_d_values) if cohens_d_values else 1.0
        for bar, p_val in zip(bars, p_values):
            if p_val < 0.001:
                sig_text = '***'
            elif p_val < 0.01:
                sig_text = '**'
            elif p_val < 0.05:
                sig_text = '*'
            else:
                sig_text = 'ns'

            y_pos = bar.get_height() + max_val * 0.05
            ax.text(bar.get_x() + bar.get_width()/2, y_pos, sig_text,
                   ha='center', va='bottom', fontweight='bold', fontsize=8)

        # Set proper limits to prevent overflow
        ax.set_ylim(0, max_val * 1.2)

    # Section 3: Clinical scales comparison
    assessment_data = CLINICAL_EDA['clinical_assessment_analysis']
    if assessment_data:
        ax = fig.add_subplot(gs[2, 0])

        scales = list(assessment_data.keys())
        asd_means = [assessment_data[scale]['asd_mean'] for scale in scales]
        non_asd_means = [assessment_data[scale]['non_asd_mean'] for scale in scales]

        x = np.arange(len(scales))
        width = 0.35

        bars1 = ax.bar(x - width/2, non_asd_means, width, label='Non-ASD',
                      color='#2E86AB', alpha=0.8)
        bars2 = ax.bar(x + width/2, asd_means, width, label='ASD',
                      color='#A23B72', alpha=0.8)

        ax.set_title('Clinical Assessment Scales\nMean Scores by ASD Status',
                    fontweight='bold', pad=10)
        ax.set_xlabel('Assessment Scales')
        ax.set_ylabel('Mean Score')
        ax.set_xticks(x)
        ax.set_xticklabels(scales, rotation=0)
        ax.legend()

    # Section 4: Redundancy analysis summary
    ax = fig.add_subplot(gs[2, 1])
    redundancy_counts = [
        len(REDUNDANCY_REPORT['correlation_analysis'].get('high_corr_pairs', [])),
        len(REDUNDANCY_REPORT['variance_analysis'].get('low_variance_features', [])),
        len(REDUNDANCY_REPORT['duplicate_analysis'].get('exact_duplicates', []))
    ]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    bars = ax.bar(['High Correlation\n(r>0.95)', 'Low Variance\n(<0.01)', 'Exact\nDuplicates'],
                 redundancy_counts, color=colors, alpha=0.8)
    ax.set_title('Feature Redundancy Analysis', fontweight='bold', pad=10)
    ax.set_ylabel('Count')

    # Add count labels with proper positioning
    max_count = max(redundancy_counts) if redundancy_counts else 1
    for bar, count in zip(bars, redundancy_counts):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max_count * 0.02,
               f'{count}', ha='center', va='bottom', fontweight='bold', fontsize=8)

    ax.set_ylim(0, max_count * 1.15)

    # Section 5: Data quality metrics
    ax = fig.add_subplot(gs[2, 2])
    quality_metrics = ['Missing Values', 'Infinite Values', 'Duplicates']
    quality_counts = [
        sum(QUALITY_REPORT[split]['missing_values'] for split in ['train', 'val', 'test']),
        sum(QUALITY_REPORT[split]['infinite_values'] for split in ['train', 'val', 'test']),
        CLINICAL_EDA['dataset_overview']['data_quality']['duplicate_rows']
    ]

    colors = ['#90EE90' if count == 0 else '#FFB6C1' for count in quality_counts]
    bars = ax.bar(quality_metrics, quality_counts, color=colors, alpha=0.8)
    ax.set_title('Data Quality Assessment\nGreen: Clean, Pink: Issues',
                fontweight='bold', pad=10)
    ax.set_ylabel('Count')

    # Add count labels with proper positioning
    max_quality = max(quality_counts) if any(quality_counts) else 1
    for bar, count in zip(bars, quality_counts):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max_quality * 0.02,
               f'{count}', ha='center', va='bottom', fontweight='bold', fontsize=8)

    ax.set_ylim(0, max_quality * 1.15)

    # Add overall figure title with proper spacing
    fig.suptitle('ASD Dataset: Publication-Ready Clinical Exploratory Data Analysis',
                fontsize=14, fontweight='bold', y=0.97)

    # Save EDA plot
    eda_path = f"{results_dir}/clinical_eda_analysis.png"
    plt.savefig(eda_path, dpi=300, bbox_inches='tight', facecolor='white',
                edgecolor='none', pad_inches=0.3)
    plt.close()

    print(f"Clinical EDA analysis saved: {eda_path}")
    return results_dir

def generate_correlation_matrix_visualization():
    """Generate professional triangular correlation matrix"""
    results_dir = create_results_directory("eda_analysis")

    # Set publication-ready style for correlation matrix
    plt.style.use('default')
    plt.rcParams.update({
        'figure.dpi': 300,
        'font.size': 9,
        'axes.titlesize': 12,
        'axes.labelsize': 9,
        'xtick.labelsize': 8,
        'ytick.labelsize': 8,
        'figure.titlesize': 14
    })

    if 'correlation_matrix' in REDUNDANCY_REPORT['correlation_analysis']:
        corr_matrix = REDUNDANCY_REPORT['correlation_analysis']['correlation_matrix']
        clinical_groups = REDUNDANCY_REPORT['clinical_analysis']['clinical_groups']

        # Create ordered feature list by clinical groups
        ordered_features = []
        for group_name in ['behavioral_core', 'assessment_scales', 'demographic', 'medical_history']:
            group_features = clinical_groups.get(group_name, [])
            group_features_in_matrix = [f for f in group_features if f in corr_matrix.columns]
            ordered_features.extend(group_features_in_matrix)

        # Add remaining features not in any group
        remaining_features = [f for f in corr_matrix.columns if f not in ordered_features]
        ordered_features.extend(remaining_features)

        if len(ordered_features) > 1 and not corr_matrix.empty:
            # Ensure ordered features exist in correlation matrix
            valid_ordered_features = [f for f in ordered_features if f in corr_matrix.columns]
            if len(valid_ordered_features) > 1:
                corr_matrix_ordered = corr_matrix.loc[valid_ordered_features, valid_ordered_features]

                # Create figure with proper sizing for triangular matrix
                fig, ax = plt.subplots(figsize=(14, 12))

                # Create triangular mask (upper triangle)
                mask = np.triu(np.ones_like(corr_matrix_ordered.values, dtype=bool), k=1)

                # Apply mask to correlation matrix
                corr_masked = corr_matrix_ordered.values.copy()
                corr_masked[mask] = np.nan

                # Create triangular correlation heatmap
                im = ax.imshow(corr_masked, cmap='RdBu_r', aspect='equal',
                              vmin=-1, vmax=1, interpolation='nearest')

                # Set ticks and labels
                n_features = len(corr_matrix_ordered.columns)
                ax.set_xticks(range(n_features))
                ax.set_yticks(range(n_features))
                ax.set_xticklabels(corr_matrix_ordered.columns, rotation=45, ha='right')
                ax.set_yticklabels(corr_matrix_ordered.index)

                # Add correlation values only for lower triangle with significance threshold
                for i in range(n_features):
                    for j in range(n_features):
                        if i > j:  # Lower triangle only
                            corr_val = corr_matrix_ordered.iloc[i, j]
                            if abs(corr_val) >= 0.3:  # Only show significant correlations
                                # Choose text color based on correlation strength
                                text_color = 'white' if abs(corr_val) > 0.7 else 'black'
                                ax.text(j, i, f'{corr_val:.2f}', ha='center', va='center',
                                       color=text_color, fontsize=8, fontweight='bold')

                # Add clinical group separators
                group_boundaries = []
                current_pos = 0
                group_labels = []

                for group_name in ['behavioral_core', 'assessment_scales', 'demographic', 'medical_history']:
                    group_features = [f for f in clinical_groups.get(group_name, [])
                                    if f in valid_ordered_features]
                    if group_features:
                        group_labels.append((group_name.replace('_', ' ').title(),
                                           current_pos, current_pos + len(group_features)))
                        current_pos += len(group_features)
                        if current_pos < len(valid_ordered_features):
                            group_boundaries.append(current_pos - 0.5)

                # Draw group separator lines
                for boundary in group_boundaries:
                    ax.axhline(y=boundary, color='white', linewidth=2.5)
                    ax.axvline(x=boundary, color='white', linewidth=2.5)

                # Add group labels on the right side
                for group_name, start_pos, end_pos in group_labels:
                    mid_pos = (start_pos + end_pos - 1) / 2
                    ax.text(n_features + 0.5, mid_pos, group_name, rotation=90,
                           ha='left', va='center', fontweight='bold', fontsize=10,
                           color='darkblue')

                # Add colorbar with proper positioning
                cbar = plt.colorbar(im, ax=ax, shrink=0.8, pad=0.05, aspect=30)
                cbar.set_label('Pearson Correlation Coefficient', rotation=270, labelpad=20)

                # Improve layout and remove upper triangle visual noise
                ax.set_xlim(-0.5, n_features - 0.5)
                ax.set_ylim(n_features - 0.5, -0.5)

                # Set title with proper spacing
                ax.set_title('Feature Correlation Matrix with Clinical Grouping\n' +
                           '(Lower Triangle: |r| >= 0.3 displayed, Upper Triangle: Hidden)',
                           fontweight='bold', pad=25)

                # Adjust layout to prevent label cutoff
                plt.tight_layout()

                # Save triangular correlation matrix
                corr_path = f"{results_dir}/triangular_correlation_matrix.png"
                plt.savefig(corr_path, dpi=300, bbox_inches='tight', facecolor='white',
                           edgecolor='none', pad_inches=0.3)
                plt.close()

                print(f"Triangular correlation matrix saved: {corr_path}")
                return results_dir

    print("Warning: Correlation matrix data not available")
    return results_dir

def generate_publication_visualizations():
    """Generate both EDA and correlation matrix as separate visualizations"""
    # Generate clinical EDA visualization
    eda_results_dir = generate_clinical_eda_visualization()

    # Generate correlation matrix visualization
    corr_results_dir = generate_correlation_matrix_visualization()

    print(f"Both visualizations saved in: {eda_results_dir}")
    return eda_results_dir

# Generate enhanced visualizations
VISUALIZATION_DIR = generate_publication_visualizations()

# ==========================================
# 9. EXPORT COMPREHENSIVE RESULTS
# ==========================================

print("\n[EXPORT] Saving comprehensive analysis results...")
print("-" * 50)

def export_comprehensive_results():
    """Export all analysis results with clinical insights (JSON-safe version)"""

    # Helper function to make objects JSON serializable
    def make_json_serializable(obj):
        """Convert complex objects to JSON-serializable format"""
        if isinstance(obj, pd.DataFrame):
            return obj.to_dict()
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, dict):
            return {key: make_json_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [make_json_serializable(item) for item in obj]
        else:
            return obj

    # Create JSON-safe copy of clinical EDA results
    clinical_eda_safe = {}
    try:
        clinical_eda_safe = make_json_serializable(CLINICAL_EDA)
    except Exception as e:
        print(f"Warning: Clinical EDA results conversion failed: {e}")
        clinical_eda_safe = {'error': 'Conversion failed', 'message': str(e)}

    # Create JSON-safe copy of redundancy report
    redundancy_safe = {}
    try:
        redundancy_temp = REDUNDANCY_REPORT.copy()

        # Handle correlation matrices separately
        if 'correlation_analysis' in redundancy_temp:
            if 'correlation_matrix' in redundancy_temp['correlation_analysis']:
                corr_matrix = redundancy_temp['correlation_analysis']['correlation_matrix']
                if isinstance(corr_matrix, pd.DataFrame):
                    redundancy_temp['correlation_analysis']['correlation_matrix'] = corr_matrix.to_dict()

        # Handle clinical analysis group correlations
        if 'clinical_analysis' in redundancy_temp:
            if 'group_correlations' in redundancy_temp['clinical_analysis']:
                for group_name, group_data in redundancy_temp['clinical_analysis']['group_correlations'].items():
                    if 'correlation_matrix' in group_data:
                        corr_matrix = group_data['correlation_matrix']
                        if isinstance(corr_matrix, pd.DataFrame):
                            redundancy_temp['clinical_analysis']['group_correlations'][group_name]['correlation_matrix'] = corr_matrix.to_dict()

        redundancy_safe = make_json_serializable(redundancy_temp)
    except Exception as e:
        print(f"Warning: Redundancy analysis conversion failed: {e}")
        redundancy_safe = {'error': 'Conversion failed', 'message': str(e)}

    comprehensive_results = {
        'timestamp': datetime.now().isoformat(),
        'cell_info': {
            'cell_number': 2,
            'cell_name': 'Enhanced EDA & Preprocessing Pipeline (Publication-Ready)',
            'methodology': 'Clinical-focused EDA with statistical rigor and IEEE visualization standards',
            'enhancements': [
                'Publication-ready clinical analysis',
                'Statistical significance testing',
                'Effect size calculations',
                'Enhanced correlation matrix with clinical grouping',
                'IEEE conference standard visualizations',
                'Preprocessed CSV generation for reproducibility'
            ]
        },
        'dataset_info': {
            'splits_loaded': {split: list(SPLITS_DATA[split].shape) for split in SPLITS_DATA.keys()},
            'ethnicity_removed': True,
            'preprocessing_completed': True,
            'clinical_focus': True,
            'preprocessed_files_saved': {
                'train': f"{PROJECT_PATHS['preprocessed']}/train_set_preprocessed.csv",
                'val': f"{PROJECT_PATHS['preprocessed']}/val_set_preprocessed.csv",
                'test': f"{PROJECT_PATHS['preprocessed']}/test_set_preprocessed.csv"
            }
        },
        'clinical_eda_results': clinical_eda_safe,
        'redundancy_analysis': redundancy_safe,
        'quality_assessment': make_json_serializable(QUALITY_REPORT),
        'preprocessing_pipeline': {
            'components': list(PREPROCESSING_PIPELINE.keys()),
            'feature_columns': PREPROCESSING_PIPELINE['feature_columns']
        },
        'processed_shapes': {
            split: {
                'features': list(PROCESSED_SPLITS[split]['X'].shape),
                'target': len(PROCESSED_SPLITS[split]['y'])
            } for split in PROCESSED_SPLITS.keys()
        },
        'publication_ready': {
            'ieee_compliant_visualizations': True,
            'statistical_rigor': True,
            'clinical_interpretation': True,
            'effect_sizes_calculated': True,
            'preprocessed_csv_saved': True
        }
    }

    # Save comprehensive results with error handling
    results_path = f"{VISUALIZATION_DIR}/comprehensive_eda_results.json"
    try:
        save_results(comprehensive_results, results_path)
        print(f"Comprehensive results saved: {results_path}")
    except Exception as e:
        print(f"Error saving comprehensive results: {e}")
        # Save a simplified version
        simplified_results = {
            'timestamp': comprehensive_results['timestamp'],
            'cell_info': comprehensive_results['cell_info'],
            'dataset_info': comprehensive_results['dataset_info'],
            'publication_ready': comprehensive_results['publication_ready'],
            'error_note': 'Full results export failed, simplified version saved'
        }
        try:
            save_results(simplified_results, results_path)
            print(f"Simplified results saved: {results_path}")
        except Exception as e2:
            print(f"Failed to save even simplified results: {e2}")

    # Save preprocessing pipeline for reuse
    pipeline_path = f"{VISUALIZATION_DIR}/preprocessing_pipeline.json"
    try:
        pipeline_export = {}
        for key, value in PREPROCESSING_PIPELINE.items():
            if key == 'feature_columns':
                pipeline_export[key] = value
            elif hasattr(value, 'get_params'):
                pipeline_export[key] = {
                    'type': type(value).__name__,
                    'params': value.get_params()
                }
            else:
                pipeline_export[key] = str(type(value))

        save_results(pipeline_export, pipeline_path)
        print(f"Preprocessing pipeline info saved: {pipeline_path}")
    except Exception as e:
        print(f"Error saving preprocessing pipeline: {e}")

    return comprehensive_results

COMPREHENSIVE_RESULTS = export_comprehensive_results()

# ==========================================
# 10. COMPLETION SUMMARY
# ==========================================

print("\n[SUMMARY] Enhanced Cell 2 Completion Summary")
print("="*80)

print("DATA PROCESSING STATUS:")
print(f"  Splits loaded: train ({SPLITS_DATA['train'].shape[0]}), val ({SPLITS_DATA['val'].shape[0]}), test ({SPLITS_DATA['test'].shape[0]})")
print(f"  Features processed: {PROCESSED_SPLITS['train']['X'].shape[1]}")
print(f"  Ethnicity bias: Removed")
print(f"  Missing values: {sum(QUALITY_REPORT[split]['missing_values'] for split in QUALITY_REPORT.keys())}")

print("\nPUBLICATION-READY CLINICAL ANALYSIS:")
behavioral_stats = CLINICAL_EDA['behavioral_features_analysis'].get('individual_statistics', {})
significant_behavioral = sum(1 for f in behavioral_stats.values() if f['statistical_test'].get('significant', False))
print(f"  Behavioral features analyzed: {len(behavioral_stats)}")
print(f"  Statistically significant features: {significant_behavioral}")
print(f"  Clinical scales analyzed: {len(CLINICAL_EDA['clinical_assessment_analysis'])}")
print(f"  Effect sizes calculated: Yes (Cohen's d)")

print("\nREDUNDANCY ANALYSIS:")
print(f"  High correlation pairs: {len(REDUNDANCY_REPORT['correlation_analysis'].get('high_corr_pairs', []))}")
print(f"  Low variance features: {len(REDUNDANCY_REPORT['variance_analysis'].get('low_variance_features', []))}")
print(f"  Clinical grouping completed: {len(REDUNDANCY_REPORT['clinical_analysis']['clinical_groups'])} groups")

print("\nENHANCED PREPROCESSING PIPELINE:")
print(f"  Method: Fit on train, transform on all splits")
print(f"  Components: {len(PREPROCESSING_PIPELINE)} pipeline components")
print(f"  Quality check: All splits processed without missing/infinite values")
print(f"  Clinical feature preservation: Yes")

print("\nPREPROCESSED CSV FILES SAVED:")
print(f"  Directory: {PROJECT_PATHS['preprocessed']}")
print(f"  Files generated:")
print(f"    - train_set_preprocessed.csv")
print(f"    - val_set_preprocessed.csv")
print(f"    - test_set_preprocessed.csv")
print(f"  Preprocessing objects saved to: {PROJECT_PATHS['preprocessors']}")

print("\nPUBLICATION OUTPUTS:")
print(f"  Results directory: {VISUALIZATION_DIR}")
print(f"  IEEE-compliant visualizations: Yes (300 DPI)")
print(f"  Statistical annotations: Yes")
print(f"  Clinical interpretations: Yes")
print(f"  JSON exports: 2 (comprehensive results + pipeline)")

print("\n" + "="*80)
print("ENHANCED CELL 2 COMPLETED SUCCESSFULLY")
print("Publication-ready processed data, clinical insights, and preprocessed CSV files generated")
print("="*80)

ASD DETECTION PROJECT: BASELINE XGBOOST IMPLEMENTATION
Cell 2: Enhanced EDA & Preprocessing Pipeline (Publication-Ready)

[VERIFICATION] Checking Cell 1 completion...
--------------------------------------------------
Cell 1 variables loaded successfully

[DATA LOADING] Loading cleaned split datasets...
--------------------------------------------------
Loaded train set: 1,270 samples x 26 features
Loaded val set: 318 samples x 26 features
Loaded test set: 397 samples x 26 features

Feature-target separation completed
Feature columns: 25

[PUBLICATION EDA] Comprehensive clinical analysis (train set only)...
--------------------------------------------------
Section 1: Dataset Overview & Quality Assessment
Section 2: Clinical Target Analysis
Section 3: Behavioral Features Analysis (A1-A10)
Section 4: Clinical Assessment Scales
Section 5: Demographic Analysis

[REDUNDANCY ANALYSIS] Comprehensive feature redundancy assessment...
--------------------------------------------------
Step 1: C